# Cleaning - Simply Recipes Ingredients List

Sources: 
* https://htmlpreview.github.io/?https://github.com/kulsoom-abdullah/kulsoom-abdullah.github.io/blob/master/AWS-lambda-implementation/model_implementation/recipe%20binary%20classification/recipe%20binary%20classification.html#Words-to-Remove
* https://towardsdatascience.com/https-towardsdatascience-com-end-to-end-recipe-cuisine-classification-e97f4ac22104


NYT - Natural Language Processor for Ingredients 

* https://github.com/nytimes/ingredient-phrase-tagger
* https://homepages.inf.ed.ac.uk/csutton/publications/crftutv2.pdf

CRF Tutorial 
* https://www.analyticsvidhya.com/blog/2018/08/nlp-guide-conditional-random-fields-text-classification/
* http://acepor.github.io/2017/03/06/CRF-Python/
* http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/
* https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html

In [61]:
import pandas as pd
import numpy as np
import re
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/markishab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
recipes_df = pd.read_pickle('../../data/02_intermediate/recipies_sr_inter.pkl')

In [63]:
recipes_df.reset_index(inplace=True, drop=True)

In [93]:
recipes_df.sort_values(by=['ingredients'])

,title,prep_time,cook_time,recipe_yield,tags,ingredients,byline,link_food
953,NaN,NaN,NaN,NaN,"['Recipe Collections', 'Budget']",[],Summer Miller,https://www.simplyrecipes.com/collection/5_eas...
1134,3 Ways to Make a Great Cheesecake,NaN,NaN,NaN,['Baking'],[],Carrie Havranek,https://www.simplyrecipes.com/cheesecake_recip...
966,15 Great Meals to Make with Canned Tuna,NaN,NaN,NaN,"['Recipe Collections', 'Dinner', 'Budget']",[],Elise Bauer,https://www.simplyrecipes.com/great_meals_to_m...
954,How to Store and Freeze Banana Bread,NaN,NaN,NaN,"['Tips', 'Banana Bread', 'Baking', 'Budget', '...",[],Summer Miller,https://www.simplyrecipes.com/how_to_store_and...
1212,The Difference Between Baking Soda and Baking ...,NaN,NaN,NaN,"['Tips', 'Baking', 'How To']",[],Elise Bauer,https://www.simplyrecipes.com/the_difference_b...
33,6 Different Meals to Make With Pulled Pork,NaN,NaN,NaN,"['Tips', 'Pork', 'Pulled Pork', 'Dinner']",[],Summer Miller,https://www.simplyrecipes.com/6_different_meal...
1154,4 Tips for Making the Best Cinnamon Rolls Ever,NaN,NaN,NaN,"['Tips', 'Baking', 'How To']",[],Summer Miller,https://www.simplyrecipes.com/4_tips_for_makin...
1290,6 Crowd-Pleasing Recipes with Cheese,NaN,NaN,NaN,"['Cheese', 'Breakfast and Brunch', 'Dessert', ...",[],Simply Recipes Team,https://www.simplyrecipes.com/6_crowd_pleasing...
889,NaN,NaN,NaN,NaN,"['Recipe Collections', 'Pasta and Noodles', 'F...",[],Carrie Havranek,https://www.simplyrecipes.com/collection/12_fr...
190,What’s the Difference Between Stock and Broth?,NaN,NaN,NaN,"['Ingredient Guides', 'Tips', 'Chicken', 'Chic...",[],Emma Christensen,https://www.simplyrecipes.com/whats_the_differ...


Let's drop the N/A columns

In [110]:
recipes_df.dropna(inplace=True)

## let's clean the list so that it can be used in a crf model 

In [111]:
ingredients_lists = list(recipes_df.ingredients)

In [112]:
len(ingredients_lists)

1177

In [113]:
ingredient_list_new = []
for recipe in ingredients_lists:
    sub_list = []
    for ingredient in recipe:
        ingredient_list_new.append(ingredient.split(','))

Let's isolate the actual ingredients

In [115]:
for element in ingredient_list_new:
    try:
        while(" " in element) : 
            element.remove(" ")
    except:
        pass

Replace ```'\\\\xa0'``` with space

In [116]:
new_list = []
for ingr_list in ingredient_list_new:
    ingr_list = [word.replace('\\\\xa0', ' ') for word in ingr_list]
    new_list.append(ingr_list)

In [117]:
len(new_list)

1177

In [118]:
final_list = []
for il in new_list:
    il = [word.replace('\\xa0', ' ') for word in il]
    final_list.append(il)

In [119]:
len(final_list)

1177

In [120]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            recipe.pop(idx)

In [121]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            print(ingredient)

In [122]:
len(final_list)

1177

In [123]:
recipes_df.ingredients[70]

["1 tablespoons extra virgin olive oil', '1 yellow onion,\\\\xa0finely chopped (about 1\\\\xa01/2 to 2 cups)', '2 cloves garlic, minced (about 2 teaspoons)', '4 ounces (110 g) Italian sweet sausage (or one 7 to 8 inch link)', '4 ounces (110 g) \\\\xa0Italian hot and spicy sausage\\\\xa0(or one 7 to 8 inch link)', '1\\\\xa0large (28 oz, 794 g) can of whole peeled tomatoes', '1 pound (16 oz, 450 g) spaghetti pasta', 'Salt"]

In [124]:
final_list[70]

["1 tablespoon extra virgin olive oil'",
 " '3 pounds pork shoulder",
 " cut into 1-2 inch chunks'",
 " 'Salt'",
 " '1 large onion",
 " chopped'",
 " '1 large carrot",
 " chopped'",
 " 'All of the cloves from 1 head garlic",
 " peeled and chopped'",
 " '1 pound hot Italian sausage links (not loose) or Spanish chorizo (use gluten-free sausages for gluten-free option)'",
 " '1 15-ounce can crushed tomatoes'",
 " '3 1/2 cups water'",
 " '2 tablespoons sweet paprika'",
 " '1 teaspoon hot paprika (can sub 1/4 teaspoon red chili flakes)'",
 " '1-2 teaspoons smoked paprika'",
 " '2 15-ounce cans chickpeas",
 ' drained and rinsed']

Make titles into list 

In [20]:
food_titles = list(recipes_df.title)

In [21]:
recipe_dict = dict()
for title, recipes in zip(food_titles, final_list):
#     print(title)
    recipe_dict[title] = recipes

In [22]:
len(recipes_df)

1748

In [25]:
recipes_df['ingredients'] = final_list

In [26]:
recipes_df.to_pickle('../../data/02_intermediate/recipes_sr_final.pickle')